In [1]:
from datasets import load_dataset

# TODO test if validation is written correctly
# TODO testing on benchmark
# TODO tracking and charting val and train loss over training
# TODO sicne we test on benchmark, we can add testing data to val or train
# TODO also check setings 'pair-class', 'pair-score','triplet', 'pair'
dataset = load_dataset("sentence-transformers/all-nli",'triplet')
print(dataset)


c:\Users\Bahno\.conda\envs\zpja_cop_project\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DatasetDict({
    train: Dataset({
        features: ['anchor', 'positive', 'negative'],
        num_rows: 557850
    })
    dev: Dataset({
        features: ['anchor', 'positive', 'negative'],
        num_rows: 6584
    })
    test: Dataset({
        features: ['anchor', 'positive', 'negative'],
        num_rows: 6609
    })
})


In [2]:
import torch
from torch import nn
from transformers import BertModel, BertTokenizer

class BertSentenceEmbedder(nn.Module):
    def __init__(self, model_name="bert-base-uncased"):
        super().__init__()
        self.bert = BertModel.from_pretrained(model_name)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        token_embeddings = outputs.last_hidden_state  # (batch, seq_len, hidden)
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size())


        sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
        token_counts = input_mask_expanded.sum(1)
        # avoid division by zero, if empty sentence
        token_counts = torch.clamp(token_counts, min=1e-9)
        return sum_embeddings / token_counts # compute mean



In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [ ]:
from torch.utils.data import DataLoader
from transformers import BertTokenizer
from torch.nn import TripletMarginLoss
loss_fn = TripletMarginLoss(margin=1.0, p=2)

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def collate_fn(batch):
    batch_size = len(batch)
    anchors = [b["anchor"] for b in batch]
    positives = [b["positive"] for b in batch]
    negatives = [b["negative"] for b in batch]

    # Combine into one list
    all_sentences = anchors + positives + negatives
    # Tokenize in one pass
    enc_all = tokenizer(all_sentences, padding=True, truncation=True, return_tensors="pt")
    # Then split back again
    enc_anchor = {k: v[:batch_size] for k, v in enc_all.items()}
    enc_pos = {k: v[batch_size:2*batch_size] for k, v in enc_all.items()}
    enc_neg = {k: v[2*batch_size:] for k, v in enc_all.items()}


    # each is dict of of 'input_ids', 'token_type_ids', 'attention_mask' which are tensors
    return enc_anchor, enc_pos, enc_neg


def validate(model,val_loader):
    model.eval()
    total_val_loss = 0
    with torch.no_grad():
        for enc_anchor, enc_pos, enc_neg in val_loader:
            input_ids_a, attn_a = enc_anchor["input_ids"], enc_anchor["attention_mask"]
            input_ids_p, attn_p = enc_pos["input_ids"], enc_pos["attention_mask"]
            input_ids_n, attn_n = enc_neg["input_ids"], enc_neg["attention_mask"]

            all_input_ids = torch.cat([input_ids_a, input_ids_p, input_ids_n], dim=0).to(device)
            all_attention_mask = torch.cat([attn_a, attn_p, attn_n], dim=0).to(device)

            all_embeddings = model(all_input_ids, all_attention_mask)
            emb_a = all_embeddings[:batch_size]
            emb_p = all_embeddings[batch_size:2*batch_size]
            emb_n = all_embeddings[2*batch_size:]

            loss = loss_fn(emb_a, emb_p, emb_n)
            total_val_loss += loss.item()

    avg_val_loss = total_val_loss / len(val_loader)
    return avg_val_loss

batch_size=16

train_loader = DataLoader(dataset["train"], batch_size=batch_size, shuffle=True, collate_fn=collate_fn,drop_last=True)
val_loader = DataLoader(dataset["dev"], batch_size=batch_size, shuffle=False, collate_fn=collate_fn,drop_last=True)

epochs = 10

model = BertSentenceEmbedder().to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)

model.train()
for epoch in range(epochs):
    total_epoch_loss = 0
    for enc_anchor, enc_pos, enc_neg in train_loader:
        # Extract input_ids and attention_mask for each
        input_ids_a, attn_a = enc_anchor["input_ids"], enc_anchor["attention_mask"]
        input_ids_p, attn_p = enc_pos["input_ids"], enc_pos["attention_mask"]
        input_ids_n, attn_n = enc_neg["input_ids"], enc_neg["attention_mask"]

        # Concatenate input_ids and attention_mask for single forward pass
        all_input_ids = torch.cat([input_ids_a, input_ids_p, input_ids_n], dim=0).to(device)
        all_attention_mask = torch.cat([attn_a, attn_p, attn_n], dim=0).to(device)

        # Forward pass
        all_embeddings = model(all_input_ids, all_attention_mask)
        # Split embeddings back into anchor, positive, negative
        emb_a = all_embeddings[:batch_size]
        emb_p = all_embeddings[batch_size:2*batch_size]
        emb_n = all_embeddings[2*batch_size:]

        # Compute triplet loss
        loss = loss_fn(emb_a, emb_p, emb_n)

        # Backprop
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_epoch_loss += loss.item()
    avg_epoch_loss = total_epoch_loss / len(train_loader)
    print(f"Epoch {epoch}: Loss = {avg_epoch_loss:.4f}")
    val_loss = validate(model,val_loader)
    print(f"Epoch {epoch+1}/{epochs} | Train Loss: {avg_epoch_loss:.4f} | Val Loss: {val_loss:.4f}")
    model.train()


Epoch 0: Loss = 0.0000


c:\Users\Bahno\.conda\envs\zpja_cop_project\Lib\site-packages\torch\nn\modules\module.py:1527: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Epoch 1/10 | Train Loss: 0.0000 | Val Loss: 0.7395
Epoch 1: Loss = 0.0000


KeyboardInterrupt: 

In [ ]:
model_path = "triplet_loss_bert.pt"
torch.save(model.state_dict(), model_path)

In [68]:
import numpy as np
from mteb.types import PromptType,BatchedInput
from mteb.abstasks.task_metadata import TaskMetadata

#TODO test it the tokenizer returns the correct shapes and etc.
# MTEB expects model with 'encode' method, that takes in list of sentences, and returns numpy embeddings
class BenchmarkEncoder:
    def __init__(self, model, tokenizer, device="cuda" if torch.cuda.is_available() else "cpu"):
        self.model = model.to(device)
        self.tokenizer = tokenizer
        self.device = device
        self.model.eval()

    @torch.no_grad()
    def encode(
        self,
        inputs: DataLoader[BatchedInput],
        task_metadata: TaskMetadata,
        hf_split: str,
        hf_subset: str,
        prompt_type: PromptType | None = None,
        **kwargs,
    ):
   
        """Encodes the given sentences using the encoder.

        Args:
            inputs: The inputs to encode.
            task_metadata: The name of the task.
            hf_subset: The subset of the dataset.
            hf_split: The split of the dataset.
            prompt_type: The prompt type to use.
            **kwargs: Additional arguments to pass to the encoder.

        Returns:
            The encoded sentences.
        """
        batch_size = kwargs['batch_size']
        
        all_embeddings = []

        for batch in inputs:
            enc = self.tokenizer(batch['text'], padding=True, truncation=True, return_tensors="pt").to(self.device)
            embeddings = self.model(enc["input_ids"], enc["attention_mask"])
            all_embeddings.append(embeddings)


        return torch.cat(all_embeddings,dim=0).cpu()


In [71]:
import mteb

wrapped = BenchmarkEncoder(model, tokenizer)

# task_types=["Clustering", "Retrieval"]
# tasks = mteb.get_tasks(tasks=["STS16"])
# tasks = mteb.get_tasks(task_types=["Clustering", "Classification","Reranking","Retrieval","STS","Sumarization"],languages=['eng'])
# tasks = mteb.get_tasks(task_types=["Clustering", "Classification","STS",],languages=['eng'])
tasks = mteb.get_tasks(task_types=['STS'],languages=['eng'])
evaluation = mteb.MTEB(tasks=tasks)
results = evaluation.run(wrapped, output_folder="mteb_results")

C:\Users\Bahno\AppData\Local\Temp\ipykernel_17376\1589277689.py:10: DeprecationWarning: MTEB is deprecated and will be removed in future versions. Please use the `mteb.evaluate` function instead.
  evaluation = mteb.MTEB(tasks=tasks)


───────────────────────────────────────────────── Selected tasks  ─────────────────────────────────────────────────

STS

- HUMESICK-R, t2t

- HUMESTS12, t2t

...

- STS17, t2t, multilingual 8 / 11 Subsets

- STS22.v2, t2t, multilingual 5 / 18 Subsets

- STSBenchmarkMultilingualSTS, t2t, multilingual 1 / 10 Subsets

- SemRel24STS, t2t, multilingual 1 / 12 Subsets

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
c:\Users\Bahno\.conda\envs\zpja_cop_project\Lib\site-packages\torch\nn\modules\module.py:1527: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSelfAttention.forward`.
  return forward_call(*args, **kwargs)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
c:\Users\Bahno\.conda\envs\zpja_cop_project\Lib\site-packages\torch\nn\modules\module.py:1527: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSelfAttention.forward`.
  return forward_call(*args, **kwargs)
Xet Storage is ena